In [7]:
import pickle
# load a file to a variable
with open('C:/ntpu_project/pickle_example.pickle', 'rb') as file:
    dicom_list = pickle.load(file)

print(f'dicom length = {len(dicom_list)}')

dicom length = 127


In [8]:
# model training
import traceback
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical

In [9]:
# prepair data for CNN
all_imgs = [np.expand_dims(dicom.standardlize(), axis=2) for dicom in dicom_list]
x = np.stack(all_imgs)
# 標記 edge 到 y 中
y = list()
for dicom in dicom_list:
    temp_y = []
    if dicom.is_same_examination_result:
        temp_y.append(1)
    else:
        temp_y.append(0)
    y.append(temp_y)
#     img_y = np.zeros(np.shape(dicom.pixel_array))
#     edge = np.array([[int(edge.x), int(edge.y)] for edge in dicom.nodule_list[0].edge_map_list])
#     img_y[tuple(edge.T)] = 1
#     img_y = np.expand_dims(img_y, axis=2)
#     y.append(img_y)
    
# for edge in dicom.nodule_list[0].edge_map_list:
#     print(y[int(edge.x), int(edge.y)])

In [10]:
# release memory
del all_imgs
del dicom_list

In [11]:
print(np.shape(x))
print(np.shape(y))
print(np.sum(y))

(127, 128, 128, 1)
(127, 1)
18


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2100, shuffle=True)
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)

In [13]:
# release memory
del x
del y

In [14]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))

(101, 128, 128, 1)
(101, 1)
(26, 128, 128, 1)
(26, 1)


In [15]:
import tensorflow.compat.v1 as tf
# import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.75, allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
tf.keras.backend.set_session(sess)

In [16]:
input_shape = np.shape(x_train[0])
print(input_shape)
batch_size = 10
epochs = 10
num_classes = 2

(128, 128, 1)


In [17]:
# 建立簡單的線性執行的模型
model = Sequential()
# 建立卷積層，filter=16,即 output space 的深度, Kernal Size: 5x5, activation function 採用 relu
model.add(Conv2D(filters=16, 
                 kernel_size=(2, 2),
                 padding="same",
                 activation='relu',
                 input_shape=input_shape))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, 
                 kernel_size=(2, 2),
                 padding="same",
                 activation='relu',
                 input_shape=input_shape))

model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))

# Flatten層把多維的輸入一維化，常用在從卷積層到全連接層的過渡。
model.add(Flatten())

# 全連接層: 128個output
model.add(Dense(128, activation='relu'))

# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))

# 使用 softmax activation function，將結果分類
model.add(Dense(num_classes, activation='softmax'))

# 編譯: 選擇損失函數、優化方法及成效衡量方式
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

print(model.summary()) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [18]:
# 進行訓練, 訓練過程會存在 train_history 變數中

train_history = model.fit(x_train, 
                          y_train_onehot,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1)

# 顯示損失函數、訓練成果(分數)
score = model.evaluate(x_test, y_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 101 samples
Epoch 1/10
101/101 [==============================] - 2s 22ms/sample - loss: 0.5274 - accuracy: 0.7921
Epoch 2/10
101/101 [==============================] - 0s 537us/sample - loss: 0.4621 - accuracy: 0.8614
Epoch 3/10
101/101 [==============================] - 0s 540us/sample - loss: 0.4420 - accuracy: 0.8515
Epoch 4/10
101/101 [==============================] - 0s 542us/sample - loss: 0.4157 - accuracy: 0.8515
Epoch 5/10
101/101 [==============================] - 0s 545us/sample - loss: 0.3461 - accuracy: 0.8515
Epoch 6/10
101/101 [==============================] - 0s 536us/sample - loss: 0.3411 - accuracy: 0.8515
Epoch 7/10
101/101 [==============================] - 0s 531us/sample - loss: 0.3235 - accuracy: 0.8812
Epoch 8/10
101/101 [==============================] - 0s 495us/sample - loss: 0.2446 - accuracy: 0.9010
Epoch 9/10
101/101 [==============================] - 0s 504us/sample - loss: 0.2350 - accuracy: 0.8812
Epoch 10/10
101/101 [=======================

In [1]:
par1={
    'Validation': 0.2, 
    'epoch': 40, 
    'Batch': 10
}
par = {
    'filter1':64,
    'filter2':128,
    'strides':1,
    'kernel': 2,
    'dropout1':0.5,
    'dropout2':0.5,
    'Neural':256
}

def DL_fun1(par,par1,x_train,y_train,x_test,y_test):
    result = None
    nf=par['filter1']
    nf1=par['filter2']
    ks=par['kernel']
    hs=np.shape(x)[1]
    ws=np.shape(x)[2]
    dr1=par['dropout1']
    dr2=par['dropout2']
    nn=par['Neural']
    vsr=par1['Validation']
    epn=par1['epoch']
    bs=par1['Batch']
    y_train_onehot = to_categorical(y_train)
    y_test_onehot = to_categorical(y_test)
    try:
        # 只使用 80% 的 GPU 記憶體config = tf.ConfigProto()
#         gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5, allow_growth=True)
#         sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
#         tf.keras.backend.set_session(sess)

        model = Sequential([
            Conv2D(filters=nf,kernel_size=(ks,ks),strides=(par['strides'],par['strides']),padding='same',input_shape=(hs,ws,1),activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(filters=nf1,kernel_size=(ks,ks),strides=(par['strides'],par['strides']),padding='same',input_shape=(hs,ws,1),activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Dropout(dr1),
            Flatten(),
            Dense(nn, activation='relu'),
            Dropout(dr2),
            Dense(3,activation='softmax'),
        ])
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        print(model.summary()) 
        train_history=model.fit(x=x_train,
                                y=y_train_onehot,
                                validation_split=vsr,
                                epochs=epn,
                                batch_size=bs,
                                verbose=2)
        scores=model.evaluate(x=x_test,y=y_test_onehot)
        result={'History':train_history, 'Score': scores}
        print(result)
    except Exception as e:
        print(traceback.format_exc())
    finally:
#         sess.close()
        pass
    return result

In [ ]:
result=DL_fun1(par,par1,x_train,y_train,x_test,y_test)

In [41]:
print(result)

None
